# Monte Carlo simulation on 3D Ising model

This is a 3D Ising model, extending from the 2D case (Ref. 2D Ising model) 


In [5]:
#----------------------------------------------------------------------#
#  Import modules
#----------------------------------------------------------------------#
import numpy as np
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from mpi4py import MPI

#----------------------------------------------------------------------#
#   Define a function myFloat transform myList to float
#----------------------------------------------------------------------#
def myFloat(myList):
        return map(float, myList)


#----------------------------------------------------------------------#
#   Monte Carlo simulation parameters
#----------------------------------------------------------------------#
SIZE = 5                          # MC size SIZE * SIZE
STEPS = 100000                     # MC steps
T = 0.1                        # MC temperature (unit: K)
J1 = 1.0                          # Nearest neighbor Heisenberg interaction parameter (unit: meV)
J2 = 1.0
J3 = J4 = 0.0

#----------------------------------------------------------------------#
#  Check periodic boundary conditions 
#----------------------------------------------------------------------#
def bc(i):
    if i+1 > SIZE-1:
        return 0
    if i-1 < 0:
        return SIZE-1
    else:
        return i


#----------------------------------------------------------------------#
#  Build system
#----------------------------------------------------------------------#
def build_system():
    #np.random.seed(1)
    system = np.random.randint(2,size=(SIZE,SIZE,SIZE))
    system[system==0] = -1
    return system

init_system = build_system()        # Initial spin order


#----------------------------------------------------------------------#
#  Calculate energy of single atom (upto second nearest interaction)
#  J1 - inplane nearest
#  J2 - out-of-plane nearest
#  J3 - inplane second nearest
#  J4 - out-of-plane second nearest
#----------------------------------------------------------------------#
def energy(system, X, Y, Z):
    eng = - system[X, Y, Z] * (
            J1 * (system[bc(X+1), Y, Z] + system[bc(X-1), Y, Z] +
                    system[X, bc(Y+1), Z] + system[X, bc(Y-1), Z]) +
            J2 * (system[X, Y, bc(Z+1)] + system[X, Y, bc(Z-1)]) +
            J3 * (system[bc(X+1), bc(Y+1), Z] + system[bc(X-1), bc(Y+1), Z] +
                    system[bc(X+1), bc(Y-1), Z] + system[bc(X-1), bc(Y-1), Z]) +
            J4 * (system[bc(X+1), Y, bc(Z+1)] + system[bc(X-1), Y, bc(Z+1)] +
                    system[bc(X+1), Y, bc(Z-1)] + system[bc(X-1), Y, bc(Z-1)] +
                    system[X, bc(Y+1), bc(Z+1)] + system[X, bc(Y-1), bc(Z+1)] +
                    system[X, bc(Y+1), bc(Z-1)] + system[X, bc(Y-1), bc(Z-1)]))
    return eng


#----------------------------------------------------------------------#                         
#   Monte carlo loops (at temperature T)                                                         
#----------------------------------------------------------------------#                                                                                                                     
def MC_loop(T):                                                                                  
    system = init_system.copy()                                                              
    n_acpt = 0                          # Number of accepted MC steps                        
                                                                                                 
    for step, x in enumerate(range(STEPS)): 
        X = np.random.randint(0,SIZE)                                                    
        Y = np.random.randint(0,SIZE)                                                    
        Z = np.random.randint(0,SIZE)                                                    
                                                                                                 
        E = -2. * energy(system, X, Y, Z)                                                
                                                                                                 
        if E <= 0.:                                                                      
            system[X, Y, Z] *= -1                                                    
            n_acpt += 1                                                              
                                                                                                 
        elif np.exp(-1./T*E) > np.random.rand():                                         
            system[X, Y, Z] *= -1                                                    
            n_acpt += 1                                                              
                                                                                                 
    return system, n_acpt                                                                    
                                                                                                 
final_system = MC_loop(T)[0]                                                                     
n_acpt = MC_loop(T)[1] 


#----------------------------------------------------------------------#
#   Monte carlo outputs
#----------------------------------------------------------------------#
def outputs():
    print '='*70
    print 'Number of accepted steps: {:6d}'.format(n_acpt)
    print 'Spin order of every MC step is save in file mc_data_3d'
    print '='*70,'\n'

    print '='*70
    print 'The initial spin order'
    print '='*70,'\n'
    print init_system,'\n'

    print '='*70
    print 'The final spinal order'
    print '='*70,'\n'
    print final_system,'\n'

outputs()

Number of accepted steps:    129
Spin order of every MC step is save in file mc_data_3d

The initial spin order

[[[ 1  1 -1 -1  1]
  [ 1 -1 -1 -1 -1]
  [-1 -1 -1  1 -1]
  [ 1  1  1  1  1]
  [ 1 -1  1  1  1]]

 [[ 1 -1  1 -1  1]
  [-1  1  1  1 -1]
  [ 1  1 -1  1  1]
  [-1  1  1 -1  1]
  [-1 -1  1 -1  1]]

 [[-1  1 -1  1 -1]
  [-1  1  1 -1 -1]
  [-1 -1 -1  1 -1]
  [ 1  1  1  1  1]
  [ 1  1 -1  1  1]]

 [[ 1  1 -1  1  1]
  [ 1  1 -1  1 -1]
  [ 1 -1  1  1 -1]
  [-1  1  1 -1  1]
  [ 1  1 -1 -1 -1]]

 [[ 1 -1  1 -1 -1]
  [-1 -1 -1  1  1]
  [-1  1  1 -1  1]
  [-1  1  1 -1 -1]
  [-1 -1 -1  1 -1]]] 

The final spinal order

[[[1 1 1 1 1]
  [1 1 1 1 1]
  [1 1 1 1 1]
  [1 1 1 1 1]
  [1 1 1 1 1]]

 [[1 1 1 1 1]
  [1 1 1 1 1]
  [1 1 1 1 1]
  [1 1 1 1 1]
  [1 1 1 1 1]]

 [[1 1 1 1 1]
  [1 1 1 1 1]
  [1 1 1 1 1]
  [1 1 1 1 1]
  [1 1 1 1 1]]

 [[1 1 1 1 1]
  [1 1 1 1 1]
  [1 1 1 1 1]
  [1 1 1 1 1]
  [1 1 1 1 1]]

 [[1 1 1 1 1]
  [1 1 1 1 1]
  [1 1 1 1 1]
  [1 1 1 1 1]
  [1 1 1 1 1]]] 



## Visualize spin configurations with 3d scattering points

In [4]:
#----------------------------------------------------------------------#
#   Plot 3d spin order
#----------------------------------------------------------------------#
fig = plt.figure()
ax = fig.gca(projection='3d')
ax.view_init(azim=80)

x, y, z = np.meshgrid(np.arange(0, SIZE, 1),
                        np.arange(0, SIZE, 1),
                        np.arange(0, SIZE, 1))

C = init_system[x, y, z]
colors = C.reshape(1,-1)[0]

ax.set_xlim(0, SIZE-1)
ax.set_ylim(0, SIZE-1)
ax.set_zlim(0, SIZE-1)

S = ax.scatter(x, y, z,
        cmap = plt.cm.seismic,
        s = 100,
        alpha = 0.6,
        edgecolors = 'none')
S.set_array(colors)

f = open("mc_data_3d","r")
out = [line.split() for line in f]
data = map(myFloat,out)
f.close()

def animate(i):
    C = np.asarray(data[i*SIZE*SIZE:(i+1)*SIZE*SIZE])
    c = C.reshape(1,-1)[0]
    S.set_array(c)
    return S

ani = animation.FuncAnimation(fig, animate, interval=V, save_count=n_acpt)
#ani.save('3d_points.gif', writer="imagemagick", fps=10, dpi=100)
#ani.save('3d_points.mp4', writer="ffmpeg", fps=10, dpi=300)
#plt.show()

<img width="640" height="480" controls src="data-and-images/3d_points.gif" />

## Parallelize the calculation with mpi4py

Futhermore, we are using mpi4py modules to paralellize the calculation on each temperature points.

Openmpi must be installed.
Use 'mpiexec -n number_of_core python file_name' to run the calculation.

Here we use one process for each temperature.

In [ ]:
#----------------------------------------------------------------------#
#  Import mpi4py module
#----------------------------------------------------------------------#
from mpi4py import MPI

#----------------------------------------------------------------------#          
#   Temperature vs. Magnetization (averaged)                                      
#                                                        
#----------------------------------------------------------------------#          
comm = MPI.COMM_WORLD                                                             
                                                                                  
size = comm.Get_size()                              # Number of cores used            
rank = comm.Get_rank()                              # Rank of the core                
                                                                                  
n_temp = 1                                          # one process for each temperature
n_tot = n_temp * size                                                             
                                                                                  
if rank == 0:                                                                     
        Temp = [1+i*1.0/2 for i in range(n_tot)]    # temp range from 1 to n_tot/2                              
else:                                                                             
        Temp = None                                                               
                                                                                  
Temp = comm.scatter(Temp, root=0)                                                 
Mag = MC_loop(Temp)                                 # use the MC_loop function defined above                             
print Temp, Mag